# REINVENT

## Title

[Improving de novo molecular design with curriculum learning](https://doi.org/10.1038/s42256-022-00494-4)

## Authors and Year

Jeff Guo, Vendy Fialková, Juan Diego Arango, Christian Margreitter, Jon Paul Janet, Kostas Papadopoulos, Ola Engkvist & Atanas Patronov (2022)

<p align="center">
   <img src="Images/AZ.png" alt="drawing" width="200"/>


## Abstract

Reinforcement learning is a powerful paradigm that has gained popularity across multiple domains. However, applying reinforcement learning may come at the cost of multiple interactions between the agent and the environment. This cost can be especially pronounced when the single feedback from the environment is slow or computationally expensive, causing extensive periods of non-productivity. Curriculum learning provides a suitable alternative by arranging a sequence of tasks of increasing complexity, with the aim of reducing the overall cost of learning. Here we demonstrate the application of curriculum learning for drug discovery. We implement curriculum learning in the de novo design platform REINVENT, and apply it to illustrative molecular design problems of different complexities. The results show both accelerated learning and a positive impact on the quality of the output when compared with standard policy-based reinforcement learning.

## Motivation

지난 번 curriculum learning 발표의 원인이 되었던 논문. Curriculum learning 전체를 공부할 자신은 없어서 이 논문에서 어떻게 적용 되었는지 만이라도 알아보고자 했습니다!

## Previous works

### REINVENT v1.0

- [2017년 발표](https://doi.org/10.1186/s13321-017-0235-x). 강화 학습을 사용한 RNN 기반의 generate model.
    - Model 구조 : GRU(Gated Recurrent Unit)
    - Pre-training : ChEMBL 1.5M structures
    - Policy based RL → task에 맞게 tuning
- Valid한 SMILES를 생성해내면서 특정 원소를 포함하지 않거나, reference compound와 유사한 물질들을 만들거나 inverse QSAR를 통해 target activity가 높은 물질들을 생성할 수 있었다고 함.

### REINVENT v2.0

- [2020년 발표](https://doi.org/10.1021/acs.jcim.0c00915).
- 변경점
    - Model 구조 : [LSTM](https://doi.org/10.1186/s13321-019-0393-0)(training set에 따라 GRU 사용하기도 함)
    - Training : GDB-13 1M/10K/1K, ChEMBL 사용. Randomized SMILES(canonical이 아니라 순서 섞인 SMILES)이 사용됨.
    - 생성물 sorting 및 filtering을 위해 [MPO score](https://doi.org/10.1021/acs.jmedchem.5b01338) 도입
    - 생성물의 다양성 확보를 위한 DF(diversity filter) 도입
        - Topological DF, identical Murcko DF, scaffole similarity DF 총 3가지 선택 가능.
        - MPO score threshold를 넘은 생성물들의 scaffold → scaffold bucket에 저장
        - 1개의 bucket의 capacity를 제한하여 그 이상의 생성물들이 들어오면 bucket에 들어오기는 하되 zero score를 줘서 penalize.
    - RL : 기본적으로는 maximizing MPO score. 목적에 맞게 reward 조절 가능.

### Dockstream

<p align="center">
   <img src="Images/Dockstream.png" alt="drawing" width="800"/>

- [2021년 발표](https://doi.org/10.1186/s13321-021-00563-7). Docking wrapper.
    - REINVENT에서 target affinity가 높은 물질의 generation을 위해 사용됨. 기존의 QSAR 방법론보다 우수한 성능을 보임.
    - Target 준비부터 docking pose 및 score 계산까지 모두 수행 가능.
    - 총 5개의 docking backend 사용 가능.
        - AutoDock Vina, GOLD, Glide, Hybrid, rDock
    - 총 3가지의 ligand embedders 사용 가능.
        - Cornia(with TautEnum), LigPrep, RDKit(with TautEnum)

### 현재 : REINVENT v3.2

## Introduction

- REINVENT에 CL implementation → 복잡한 reward function 사용시 resource 절약 및 성능 향상.

## CL set-up

- Complex task를 작은 constituent task로 분해함.
- Production objective에 맞춰 점점 복잡하게 task를 배열하여 학습 진행.

## Results

### Target scaffold construction

<p align="center">
   <img src="Images/Scaffold.png" alt="drawing" width="800"/>

- Production objective : target scaffold를 갖는 compounds의 생성.
- Scoring function : target scaffold 유무.
    - Matching sequential substructures; 생성물에 target scaffold 있다 → 1.0, 없다 → 0.5
    - 전체 generated compounds의 score의 평균을 냄.
- Standard policy-based RL : 2,000 epoch가 지나도 평균 점수가 0.5를 넘지 않음.
    - Standard policy-based RL에는 너무 복잡한 task…!
- CL
    - Target scaffold를 6개로 설정. 앞의 5개 target scaffold는 최종 target scaffold의 substructure.
    - 구조가 단순한 substructure부터 시작해서 생성물의 matching score 계산. 모델의 score 평균이 0.8에 도달하면 다음 task로 넘어가며, 최종적으로 target scaffold에 대한 task 진행.
    - 결과적으로 standard policy-based RL 대비 빠른 시간에 원하는 성능을 얻어낼 수 있었음.

### Generate compounds that are drug-like and satisfy a molecular docking constraint

<p align="center">
   <img src="Images/Docking.png" alt="drawing" width="600"/>

- Production objective : Ala 162와 2개의 수소 결합을 하면서 reference ligand보다 docking score가 좋고, 그러면서도 QED가 너무 낮지 않은 compounds의 생성.
- Scoring function : docking score와 QED의 가중합
    - 논문에 따로 기술되어있지 않으며, supplementary information의 실제 score를 역산하여 알아냄. (Special thanks to Tyler)
- Standard policy-based RL : 첫 100 epoch 동안 거의 0점에 머무르다가 200 epoch 넘어가면서 슬슬 score가 높아짐.
- CL
    - Scoring function 수정. 기존 docking score와 QED scoring에 similarity 추가.
        - 2D similarity : Tanimoto, 3D similarity : ROCS
        - low : 0.5, high : 0.8 for Tanimoto and 0.75 for ROCS
    - 첫 번째 생성시에는 reference ligand와의 similarity가 높은 물질을 생성하도록 함.
    - 두 번째 이후로 수정된 scoring function을 기반으로 학습 진행.
    - 결과는 보시는대로… 좋았다!
- 이 밖에도 CL 적용시 Reference ligand보다 좋은 docking score를 얻은 물질들을 더 많이 만들어냈고, 생성된 scaffold가 더 많았고 어쩌고 저쩌고… 하여튼 좋았다고 합니다.

### 총평

1. Curriculum에서 생각보다 heuristic한 지점이 많은 것 같습니다. **Domain 지식을 기반**으로 이러이러 했을 때 성능이 좋을거야! 라고 가정하고 curriculum을 짠 느낌이었습니다.
    - 저희 모델의 성능 향상에 도움이 될게 있을 지 머리를 열심히 굴려보도록 하겠습니다..
2. MIN-T로 좋은 lead compound가 발굴된다면 그 뒷 단계에서 REINVENT를 그대로 사용해봐도 괜찮을 것 같다 생각했습니다.
    - **Lead optimization** 과정에서 좋은 결과를 낼 것 같은 느낌…?
3. 지난 번 CL에 대해 공부했던 것이 그래도 콩알만큼 논문을 이해하는 데 도움이 된 것 같아 개인적으로 뿌듯…ㅎ
4. 다음번에는 [디퓨-전 molecule generation](https://doi.org/10.48550/arXiv.2209.00865)으로 돌아오겠읍니다. ~~(Demian 살려주세요)~~